In [2]:
import os
import json
import pandas as pd

from tqdm import tqdm
from langchain import PromptTemplate

from src.DST.evaluate_utils import remapping
from src.DST.dst import SLOTS_DESCRIPTIONS
from src.config import CONFIG

from dataclasses import dataclass, field
from typing import Optional
from transformers import TrainingArguments


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to utilize.
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={"help": "The path of the HuggingFace model."}
    )
    use_int8: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use int8 model or not."}
    )
    use_deepspeed: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use deepspeed model or not."}
    )
    

@dataclass
class DataArguments:
    """
    Arguments pertaining to the data loading and preprocessing pipeline.
    """
    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "Train dataset path"}
    )
    dataset_names: Optional[str] = field(
        default=None,
        metadata={"help": "Train dataset paths"}
    )
    root_data_path: Optional[str] = field(
        default="./data", metadata={"help": "The path to the data directory."},
    )
    mwoz_path: Optional[str] = field(
        default="/home/willy/instructod/MultiWOZ_2.1/",
        metadata={"help": "MWOZ path"}
    )
    dialog_history_limit_dst: Optional[int] = field(
        default=0,
        metadata={"help": "Lenght of dialogue history for dst"}
    )
    dialog_history_limit_dst_recorrect: Optional[int] = field(
        default=0,
        metadata={"help": "Lenght of dialogue history for dst update"}
    )
    dialog_history_limit_rg: Optional[int] = field(
        default=20,
        metadata={"help": "Lenght of dialogue history for response generation"}
    )
    dialog_history_limit_e2e: Optional[int] = field(
        default=20,
        metadata={"help": "Lenght of dialogue history for e2e"}
    )
    single_domain_only: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to keep only the single domain sample or not"}
    )
    with_slot_description: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use slot description or not for DST"}
    )
    with_req_inf_differentiation: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to differentiate between require and inform slot for DST"}
    )
    with_all_slots: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to use all slots or not"}
    )

@dataclass
class PromptingArguments(TrainingArguments):
    """
    Arguments pertraining to the prompting pipeline.
    """
    output_dir: Optional[str] = field(
        default="./out",
        metadata={"help": "Output directory"},
    )
    max_requests_per_minute: Optional[int] = field(
        default=20,
        metadata={"help": "Max number of requests for OpenAI API."}
    )
    openai_api_key_name: Optional[str] = field(
        default="OPENAI_API_KEY",
        metadata={"help": "OpenAI API key name."}
    )



In [3]:
data_args = DataArguments()

In [4]:
data_args.dialog_history_limit_e2e = -1
data_args.dialog_history_limit_rg = -1

In [16]:
class PromptConstructor():
    def __init__(self, 
                 config):
        self.config = config
        self.instructions = config["INSTRUCTIONS"]
        self.prompt_templates = config["PROMPT_TEMPLATES"]
        
    def _get_slots_from_domains(self, domains, with_slot_description, with_req_inf_differentiation, with_all_slots):
        # slot_description = self.config["slot_descrpition"]
        if with_all_slots:
            domains = "all"
        
        if with_slot_description:
            with_req_inf_differentiation = False #Slot description is the discriminator

        if domains == "all":
            if with_req_inf_differentiation:
                req_slots = ", ".join(self.config["multiwoz21"]["all_requestable_slots"])
                inf_slots = ", ".join(self.config["multiwoz21"]["all_informable_slots"])
            else:
                slots = set(self.config["multiwoz21"]["all_requestable_slots"] + 
                            self.config["multiwoz21"]["all_informable_slots"])
                slots = ", ".join(slots)
        elif not isinstance(domains, list):
            raise ValueError("""Provided domain should be either 'all' or list of valid domain names:
                                - for multiwoz2.1 and 2.4: taxi, restaurant, hotel, train, attraction 
                                - for SGD: To-do""")
        else:
            req_slots = ""
            inf_slots = ""
            domain_req_slots = []
            domain_inf_slots = []
            for domain in domains:
                domain_req_slots += self.config["multiwoz21"]["requestable_slots"][domain]
                domain_inf_slots += self.config["multiwoz21"]["informable_slots"][domain]
            if with_req_inf_differentiation:
                domain_req_slots = set(domain_req_slots)
                domain_inf_slots = set(domain_inf_slots)
                req_slots += ", ".join(domain_req_slots)
                inf_slots += ", ".join(domain_inf_slots)
            else:
                slots = set(domain_req_slots + domain_inf_slots)
                slots = ", ".join(slots)

        if with_req_inf_differentiation:
            slots_info = f"Requestable slots: {req_slots}\nInformable slots: {inf_slots}"
        else:
            slots_info = f"{slots}"

        if with_slot_description:
            slots = slots.split(", ")
            slots_info = ""
            for slot in slots:
                if slot not in self.config["multiwoz21"]["all_informable_slots"]:
                    continue
                slots_info += f"name: {slot}, description: {SLOTS_DESCRIPTIONS[slot]}\n"
            slots_info = slots_info[:-2]
        
        return slots_info
    
    
    def _build_prompt(self, mode="", dialogue_context="", ontology="", slots="", dialogue_acts="", belief_states="", database=""):
        prompt = ""
        if mode == "dst":
            instruction = self.instructions["instruction_with_slots"]
            template_variables = self.prompt_templates["template_with_slots"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     slots=slots,
                                     dialogue_context=dialogue_context)
            
        elif mode == "dst_recorrect":
            instruction = self.instructions["instruction_with_slots_recorrect"]
            template_variables = self.prompt_templates["template_with_slots_recorrect"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])            
            prompt = template.format(instruction=instruction,
                                    slots=slots,
                                    dialogue_context=dialogue_context,
                                    belief_states=belief_states)
            
        elif mode == "database_query":
            instruction = self.instructions["instruction_query_database"]
            template_variables = self.prompt_templates["template_query_database"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     belief_states=belief_states)
            
        elif mode == "response_generation":
            example = self.config["EXAMPLES"]["response_generation"]
            
            instruction = self.instructions["instruction_response_generation"]
            template_variables = self.prompt_templates["template_response_generation"]
            template = PromptTemplate(input_variables = template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     example=example,
                                     dialogue_context=dialogue_context)
        elif mode == "e2e":
            instruction = self.instructions["instruction_e2e"]
            template_variables = self.prompt_templates["template_e2e"]
            template = PromptTemplate(input_variables = template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     database=database,
                                     dialogue_context=dialogue_context)

        else:
            raise ValueError("'mode' should be one of: [dst, dst_recorrect, database_query, response_generation, e2e]")
        
        return prompt


class MWOZ_Dataset(PromptConstructor):
    def __init__(self,
                 config,
                 data_args):
        PromptConstructor.__init__(self, config)
        self.dataset = {"id":[],
                        "dialogue_id":[],
                        "dialogue_context":[],
                        "turn":[],
                        "prompt_dst":[],
                        "prompt_dst_update":[],
                        "prompt_rg":[],
                        "prompt_e2e":[],
                        "domains":[],
                        "turn_domain":[],
                        "gold_turn_bs":[],
                        "gold_bs":[],
                        "gold_act":[],
                        "gold_response":[],
                        "gold_database_result":[],
                        }
        
        print("Loading data...")
        self.all_data, self.testfiles, self.system_acts = self._get_mwoz_data(data_args.mwoz_path)
        print("Loading databases...")
        self.dbs_lexicalized = self._get_dbs_lexicalized(data_args.mwoz_path)
        self.idx = 0
        self.dialog_history_limit_dst = data_args.dialog_history_limit_dst
        self.dialog_history_limit_rg = data_args.dialog_history_limit_rg
        self.dialog_history_limit_e2e = data_args.dialog_history_limit_e2e
        self.single_domain_only = data_args.single_domain_only
        self.with_slot_description = data_args.with_slot_description
        self.with_req_inf_differentiation = data_args.with_req_inf_differentiation
        self.with_all_slots = data_args.with_all_slots
        self.all_domains = ["restaurant", "taxi", "hotel", "train", "attraction"]

        print("Processing mwoz...")
        for sample in tqdm(self.all_data):
            if sample in self.testfiles:
                dialogue_log = self.all_data[sample]["log"]
                self._process_dialogue_log(sample=sample,
                                           dialogue_log=dialogue_log)

        self.dataset = pd.DataFrame(self.dataset)
        if self.single_domain_only:
            for index, row in tqdm(self.dataset.iterrows()):
                if len(row["domains"]) != 1:
                    self.dataset.drop(index, inplace=True)

                    
    def _get_mwoz_data(self, mwoz_path):
        data_path = os.path.join(mwoz_path, "data.json")
        testListFile_path = os.path.join(mwoz_path, "testListFile.txt")
        system_acts_path = os.path.join(mwoz_path, "system_acts.json")

        with open(data_path, "r") as f:
            all_data = json.load(f)
            
        with open(testListFile_path, "r") as f:
            testfiles = f.read()
        testfiles = testfiles.split("\n")
        
        with open(system_acts_path, "r") as f:
            system_acts = json.load(f)
            
        return all_data, testfiles, system_acts
    
    def _get_dbs_lexicalized(self, mwoz_path):
        domains = ["restaurant", "hotel", "train", "attraction"]
        keep_data = {"restaurant":["address", "area", "food", "name", "pricerange", "phone", "postcode"],
                    "attraction":["name", "area", "address", "type", "postcode"],
                    "hotel":["name", "address", "area", "phone", "postcode", "pricerange", "stars"],
                    "train":["departure", "destination"]}
        dbs_lexicalized = {}
        for domain in domains:
            db_path = os.path.join(mwoz_path, f"{domain}_db.json")
            with open(db_path, "r") as f:
                db_data = json.load(f)

            db_lexicalized = []
            for row in db_data:
                row_keep = []
                for key in keep_data[domain]:
                        if key in row:
                            row_keep.append(f"{key}: {row[key]}")
                db_lexicalized.append(", ".join(row_keep))
            dbs_lexicalized[domain] = "\n".join(set(db_lexicalized))

        return dbs_lexicalized
    
    def _process_dialogue_log(self, sample, dialogue_log):

        dialog_history_memory_dst = []
        dialog_history_memory_rg = []
        dialog_history_memory_e2e = []
        dialog_history_dst = ""
        dialog_history_rg = ""
        dialog_history_e2e = ""
        turn_domain = ""
        domains = self._get_domains_from_log(dialogue_log)
        slots = self._get_slots_from_domains(domains, 
                                             self.with_slot_description,
                                             self.with_req_inf_differentiation,
                                             self.with_all_slots) # or all

        for turn_nb, turn in enumerate(dialogue_log):

            if turn_nb % 2 == 0:
                speaker = "USER"
            else:
                speaker = "SYSTEM"
            
            utterance = f"""{speaker}: {turn["text"]}\n"""
            dialog_act = turn["dialog_act"]
            
            dialogue_context_dst = dialog_history_dst + utterance
            prompt_dst = self._build_prompt(mode="dst",
                                            slots=slots,
                                            dialogue_context=dialogue_context_dst)
            
            lexicalized_act = self._lexicalize_act(dialog_act)
            dialogue_context_rg = dialog_history_rg + utterance + f"ACT:{lexicalized_act}\nSYSTEM:"
            prompt_rg = self._build_prompt(mode="response_generation",
                                            dialogue_context=dialogue_context_rg)
            
            dialogue_context_e2e = dialog_history_e2e + utterance + "SYSTEM:"
            # need to have utterance level domain here
            cur_system_act = self.system_acts[sample.split(".")[0]][str((turn_nb//2)+1)]
            turn_domain = self._get_domain_from_turn(turn_domain, cur_system_act)
            if turn_domain and turn_domain != "taxi":
                database = self.dbs_lexicalized[turn_domain]
            else:
                database = ""
            prompt_e2e = self._build_prompt(mode="e2e",
                                            database=database,
                                            dialogue_context=dialogue_context_e2e)

            dialog_history_dst, dialog_history_memory_dst = self._update_dialogue_memory(utterance, 
                                                                                         dialogue_log, 
                                                                                         self.dialog_history_limit_dst, 
                                                                                         dialog_history_memory_dst)
            dialog_history_rg, dialog_history_memory_rg = self._update_dialogue_memory(utterance, 
                                                                                       dialogue_log, 
                                                                                       self.dialog_history_limit_rg,
                                                                                       dialog_history_memory_rg)
            dialog_history_e2e, dialog_history_memory_e2e = self._update_dialogue_memory(utterance, 
                                                                                         dialogue_log, 
                                                                                         self.dialog_history_limit_e2e, 
                                                                                         dialog_history_memory_e2e) 
                
            metadata = turn["metadata"]
            bspn_dict = {}
            if metadata:
                for domain in metadata:
                    slot_values = metadata[domain]["semi"]
                    for slot in slot_values:
                        value = slot_values[slot]
                        if value and value not in ["not mentioned", "none"]:
                            if domain in bspn_dict:
                                bspn_dict[domain].append(remapping(slot))
                                bspn_dict[domain].append(remapping(value))
                            else:
                                bspn_dict[domain] = [remapping(slot), remapping(value)]
                bspn = " ".join([f"[{domain}] {' '.join(bspn_dict[domain])}" for domain in bspn_dict])

            self.idx += 1
            if turn_nb % 2 == 0:
                self.dataset["gold_turn_bs"].append(dialog_act)
                self.dataset["dialogue_context"].append(dialogue_context_dst)
                self.dataset["gold_database_result"].append(None) 
                self.dataset["turn"].append(turn_nb//2)
                self.dataset["domains"].append(domains)
                self.dataset["id"].append(self.idx//2)
                self.dataset["dialogue_id"].append(sample)
                self.dataset["prompt_dst"].append(prompt_dst)
                self.dataset["prompt_dst_update"].append(prompt_dst)
                self.dataset["prompt_rg"].append(prompt_rg)
                self.dataset["prompt_e2e"].append(prompt_e2e)
                self.dataset["turn_domain"].append(turn_domain)
            else:
                self.dataset["gold_response"].append(utterance)
                self.dataset["gold_bs"].append(bspn)
                self.dataset["gold_act"].append(dialog_act)

    def _update_dialogue_memory(self, utterance, dialogue_log, dialog_history_limit, dialog_history_memory):
        if dialog_history_limit != 0:
            if dialog_history_limit == -1:
                dialog_history_limit = len(dialogue_log)
            if len(dialog_history_memory) >= dialog_history_limit:
                dialog_history_memory.pop(0)
            dialog_history_memory.append(utterance)

        dialog_history = "".join(dialog_history_memory)
        return dialog_history, dialog_history_memory
    
    def _lexicalize_act(self, act):
        lexicalized_acts = []
        lexicalize_mapping = {"leave": "leave time",
                              "arrive":"arrival time",
                              "departure":"departure place",
                              "post":"postcode",
                              "addr":"address"}

        for act, slot_values in act.items():


            if "request" in act.lower():
                requests = []
                for (slot, value) in slot_values:
                    slot = slot.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        requests.append(slot)
                if requests:
                    lexicalized_act = "Request the user about " + ", ".join(requests) + "."
                    lexicalized_acts.append(lexicalized_act)

            elif "recommend" in act.lower():
                recommends = []
                for (slot, value) in slot_values:
                    slot, value = slot.lower(), value.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        recommends.append(value)
                if recommends:
                    lexicalized_act = "Recommend the user for " + ", ".join(recommends) + "."
                    lexicalized_acts.append(lexicalized_act)

            elif "inform" in act.lower():
                informs = []
                for (slot, value) in slot_values:
                    slot, value = slot.lower(), value.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        informs.append(f"the {slot} is {value}")
                if informs:
                    lexicalized_act = "Inform the user that " + ", ".join(informs) + "."  
                    lexicalized_acts.append(lexicalized_act)

            else:
                pass
        if lexicalized_acts:
            return " ".join(lexicalized_acts)
        else:
            return "None"
        
    def _get_domain_from_turn(self, domain, act):
        for k in act:
            turn_domain = k.lower().split("-")[0]
            if turn_domain in self.all_domains:
                return turn_domain
        return domain
            

    def _get_domains_from_log(self, dialogue_log):
        domains = []
        for log in dialogue_log:
            for domain_act in log["dialog_act"]:
                domain = domain_act.split("-")[0].lower()
                if domain in self.all_domains and domain not in domains:
                    domains.append(domain)
        return domains
                

In [17]:
mwoz = MWOZ_Dataset(CONFIG, data_args)
dataset = mwoz.dataset
# for i in range(10):
#     for key in dataset:
#         print(f"""{key}: {dataset[key][i]}""")
#     print("================")

Loading data...
Loading databases...
Processing mwoz...


100%|███████████████████████████████████████████████████████████████████████████████| 10438/10438 [00:02<00:00, 3781.57it/s]


In [23]:
dataset.head(20)

,id,dialogue_id,dialogue_context,turn,prompt_dst,prompt_dst_update,prompt_rg,prompt_e2e,domains,turn_domain,gold_turn_bs,gold_bs,gold_act,gold_response,gold_database_result
0,0,SNG0073.json,USER: I would like a taxi from Saint John's co...,0,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'Taxi-Inform': [['Dest', 'pizza hut fen ditto...",[taxi] dest pizza hut fenditton depart saint j...,"{'Taxi-Request': [['Leave', '?'], ['Arrive', '...",SYSTEM: What time do you want to leave and wha...,None
1,1,SNG0073.json,USER: I want to leave after 17:15.\n,1,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'Taxi-Inform': [['Leave', '17:15']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'Taxi-Inform': [['Car', 'blue honda'], ['Phon...",SYSTEM: \nBooking completed! your taxi will be...,None
2,2,SNG0073.json,USER: Thank you for all the help! I appreciate...,2,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'general-thank': [['none', 'none']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'general-reqmore': [['none', 'none']]}",SYSTEM: You are welcome. Is there anything el...,None
3,3,SNG0073.json,"USER: No, I am all set. Have a nice day. Bye.\n",3,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'general-bye': [['none', 'none']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'general-bye': [['none', 'none']]}",SYSTEM: you too! thank you\n,None
4,4,PMUL4648.json,USER: Please find a restaurant called Nusha.\n,0,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[restaurant, attraction]",restaurant,"{'Restaurant-Inform': [['none', 'none']], 'Att...",,"{'Restaurant-Request': [['Food', '?']], 'Resta...",SYSTEM: I don't seem to be finding anything ca...,None
5,5,PMUL4648.json,USER: I am not sure of the type of food but co...,1,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[restaurant, attraction]",restaurant,"{'general-thank': [['none', 'none']]}",,"{'Restaurant-Request': [['Name', '?']], 'Resta...",SYSTEM: Could you double check that you've spe...,None
6,6,PMUL4648.json,"USER: It's not a restaurant, it's an attractio...",2,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[restaurant, attraction]",attraction,"{'Attraction-Inform': [['Name', 'nusha']]}",[attraction] name nusha,"{'general-reqmore': [['none', 'none']], 'Attra...",SYSTEM: oh its okay. that is an entertainment ...,None
7,7,PMUL4648.json,"USER: No, but please confirm their address aga...",3,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[restaurant, attraction]",attraction,"{'Attraction-Request': [['Addr', '?'], ['Post'...",[attraction] name nusha,"{'general-reqmore': [['none', 'none']], 'Attra...","SYSTEM: Their address is Unit g6, Cambridge Le...",None
8,8,PMUL4648.json,USER: I want Indian food in 

In [19]:
print(dataset["prompt_dst_update"][1])

Generate the belief state of the very last dialogue turn in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. The results should be in json format following this format: {'slot1':'value1', 'slot2':'value2', etc...}. Use the slot from SLOTS to generate the belief state:

SLOTS:
car, food, people, address, pricerange, name, postcode, price, parking, stars, destination, internet, phone, type, id, arriveby, time, area, day, reference, leaveat, departure, stay

CONTEXT:
USER: I want to leave after 17:15.



In [24]:
print(dataset["prompt_rg"][15])

In a task oriented dialogue setting, generate a SYSTEM response to the USER query in the conversation provided in CONTEXT. You should follow the information provided in ACT to generate this answer. Do not answer with anything other than what is provided in the dialogue act:

You can follow this example:
USER: How much does the banana cost?
ACT: Inform the user that the price is 10$, the promotion is 80%, and the choice is 5, and request the amount that user wants.
SYSTEM: There are 5 to choose from. Banana currently cost 10$, with a 80% off. How many would you like?

CONTEXT:
USER: I want to find a moderately priced restaurant. 
SYSTEM: I have many options available for you! Is there a certain area or cuisine that interests you?
USER: Yes I would like the restaurant to be located in the center of the attractions. 
ACT:Inform the user that the area is centre.
SYSTEM:


In [21]:
print(dataset["turn_domain"][3])

taxi


In [22]:
print(dataset["prompt_e2e"][3])

Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one. Use the following knowledge base to interact with the user:



USER: I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
SYSTEM: What time do you want to leave and what time do you want to arrive by?
USER: I want to leave after 17:15.
SYSTEM: 
Booking completed! your taxi will be blue honda Contact number is 07218068540
USER: Thank you for all the help! I appreciate it.
SYSTEM: You are welcome.  Is there anything else I can help you with today?
USER: No, I am all set.  Have a nice day.  Bye.
SYSTEM:


In [49]:
rg_results = pd.read_csv("src/RG/results/gpt-3.5-turbo_rg_output.csv")

In [50]:
for idx, row in rg_results.iterrows():
    print("prompt", row["prompt_rg"])
    print("----")
    print("gold_act", row["gold_act"])
    print("----")
    print("gold", row["gold_response"])
    print("----")
    print("pred", row["preds"])
    print("=======")

prompt In a task oriented dialogue setting, generate a SYSTEM response to the USER query in the conversation provided in CONTEXT. You should follow the information provided in ACT to generate this answer. Do not answer with anything other than what is provided in the dialogue act:

You can follow this example:
USER: How much does the banana cost?
ACT: Inform the user that the price is 10$, the promotion is 80%, and the choice is 5, and request the amount that user wants.
SYSTEM: There are 5 to choose from. Banana currently cost 10$, with a 80% off. How many would you like?

CONTEXT:
USER: I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
ACT:Request the user about leave time, arrival time.
SYSTEM:
----
gold_act {'Taxi-Request': [['Leave', '?'], ['Arrive', '?']]}
----
gold SYSTEM: What time do you want to leave and what time do you want to arrive by?

----
pred What time would you like to leave Saint John's college and what time do you need to arrive at Pizza Hut Fe

In [38]:
dataset

,id,dialogue_id,dialogue_context,turn,prompt_dst,prompt_dst_update,prompt_rg,prompt_e2e,domains,turn_domain,gold_turn_bs,gold_bs,gold_act,gold_response,gold_database_result
0,0,SNG0073.json,USER: I would like a taxi from Saint John's co...,0,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'Taxi-Inform': [['Dest', 'pizza hut fen ditto...",[taxi] dest pizza hut fenditton depart saint j...,"{'Taxi-Request': [['Leave', '?'], ['Arrive', '...",SYSTEM: What time do you want to leave and wha...,None
1,1,SNG0073.json,USER: I want to leave after 17:15.\n,1,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'Taxi-Inform': [['Leave', '17:15']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'Taxi-Inform': [['Car', 'blue honda'], ['Phon...",SYSTEM: \nBooking completed! your taxi will be...,None
2,2,SNG0073.json,USER: Thank you for all the help! I appreciate...,2,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'general-thank': [['none', 'none']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'general-reqmore': [['none', 'none']]}",SYSTEM: You are welcome. Is there anything el...,None
3,3,SNG0073.json,"USER: No, I am all set. Have a nice day. Bye.\n",3,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,[taxi],taxi,"{'general-bye': [['none', 'none']]}",[taxi] leave 17:15 dest pizza hut fenditton de...,"{'general-bye': [['none', 'none']]}",SYSTEM: you too! thank you\n,None
4,4,PMUL4648.json,USER: Please find a restaurant called Nusha.\n,0,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[restaurant, attraction]",restaurant,"{'Restaurant-Inform': [['none', 'none']], 'Att...",,"{'Restaurant-Request': [['Food', '?']], 'Resta...",SYSTEM: I don't seem to be finding anything ca...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7367,7367,PMUL3688.json,USER: any of your choice. get me the address a...,1,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[attraction, train]",attraction,"{'Attraction-Request': [['Fee', '?'], ['Addr',...",[attraction] area centre,"{'Attraction-Recommend': [['Addr', 'Market Str...",SYSTEM: The Holy Trinity Church is located on ...,None
7368,7368,PMUL3688.json,USER: I also need a train out to the airport o...,2,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[attraction, train]",train,"{'Train-Inform': [['Day', 'friday']]}",[attraction] area centre [train] dest stansted...,"{'Train-Request': [['Depart', '?'], ['Dest', '...","SYSTEM: Where will you be departing from, and ...",None
7369,7369,PMUL3688.json,USER: leaving cambridge and going to stansted ...,3,Generate the belief state of the very last dia...,Generate the belief state of the very last dia...,"In a task oriented dialogue setting, generate ...",Generate the answer of the SYSTEM in the follo...,"[attraction, train]",train,{},[attraction] area centre [train] leave 08:45 d...,"{'Train-Inform': [['Leave', '9:40']]}",SYSTEM: There is a train t

In [68]:
e2e_results = pd.read_csv("src/e2e/results/gpt-3.5-turbo_e2e_output.csv")

In [69]:
for idx, row in e2e_results.iterrows():
    print("prompt", row["prompt_e2e"])
    print("----")
    print("gold", row["gold_response"])
    print("----")
    print("pred", row["preds"])
    print("=======")

prompt Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one, and generate nothing other that the SYSTEM reply. Use the following knowledge base to interact with the user:

USER: I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
SYSTEM:
----
gold SYSTEM: What time do you want to leave and what time do you want to arrive by?

----
pred What time would you like the taxi to arrive?
prompt Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide mul

In [52]:
def completion(prompt):
    import openai
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    response = completion.choices[0].message.content.strip()
    return response

In [57]:
prompt = dataset["prompt_e2e"][4]

In [92]:
prompt = """Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one, and generate nothing other that the SYSTEM reply. Use the following knowledge base in csv format to interact with the user:

name: abbey pool and astroturf pitch, area: east, type: swimmingpool, postcode: cb58nt
name: adc theatre, area: centre, type: theatre, postcode: cb58as
name: all saints church, area: centre, type: architecture, postcode: cb58bs
name: ballare, area: centre, type: nightclub, postcode: cb23na
name: broughton house gallery, area: centre, type: museum, postcode: cb11ln
name: byard art, area: south, type: museum, postcode: cb21sj
name: cafe jello gallery, area: west, type: museum, postcode: cb30af
name: camboats, area: east, type: boat, postcode: cb58sx
name: cambridge and county folk museum, area: west, type: museum, postcode: cb30aq
name: cambridge arts theatre, area: centre, type: theatre, postcode: cb23pj
name: cambridge artworks, area: east, type: museum, postcode: cb13ef
name: cambridge book and print gallery, area: west, type: museum, postcode: cb39ey
name: cambridge contemporary art, area: centre, type: museum, postcode: cb21su
name: cambridge museum of technology, area: east, type: museum, postcode: cb58ld
name: cambridge university botanic gardens, area: centre, type: park, postcode: cb21jf
name: castle galleries, area: centre, type: museum, postcode: cb23bj
name: cherry hinton hall and grounds, area: east, type: entertainment, postcode: cb18dw
name: cherry hinton water play, area: east, type: park, postcode: cb18dw
name: christ's college, area: centre, type: college, postcode: cb23bu
name: churchill college, area: west, type: college, postcode: cb30ds
name: cineworld cinema, area: south, type: cinema, postcode: cb17dy
name: clare college, area: west, type: college, postcode: cb21tl
name: clare hall, area: west, type: college, postcode: cb39al
name: club salsa, area: centre, type: nightclub, postcode: cb12jb
name: corpus christi, area: centre, type: college, postcode: cb21rh
name: downing college, area: centre, type: college, postcode: cb21dq
name: emmanuel college, area: centre, type: college, postcode: cb23ap
name: funky fun house, area: east, type: entertainment, postcode: cb58hy
name: gallery at twelve a high street, area: east, type: museum, postcode: cb15dh
name: gonville and caius college, area: centre, type: college, postcode: cb21ta
name: great saint mary's church, area: centre, type: architecture, postcode: cb23pq
name: holy trinity church, area: centre, type: architecture, postcode: cb23nz
name: hughes hall, area: centre, type: college, postcode: cb12ew
name: jesus college, area: centre, type: college, postcode: cb58bl
name: jesus green outdoor pool, area: north, type: swimmingpool, postcode: cb43px
name: kambar, area: centre, type: nightclub, postcode: cb23qb
name: kettle's yard, area: west, type: museum, postcode: cb30aq
name: king's college, area: centre, type: college, postcode: cb21st
name: kings hedges learner pool, area: north, type: swimmingpool, postcode: cb42xh
name: little saint mary's church, area: centre, type: architecture, postcode: cb21qy
name: lynne strover gallery, area: west, type: museum, postcode: cb30aq
name: magdalene college, area: west, type: college, postcode: cb30ag
name: milton country park, area: north, type: park, postcode: cb46az
name: mumford theatre, area: centre, type: theatre, postcode: cb11pt
name: museum of archaelogy and anthropology, area: centre, type: museum, postcode: cb23dz
name: museum of classical archaeology, area: west, type: museum, postcode: cb39da
name: Nusha, area: south, type: entertainment, postcode: cb17dy
name: old schools, area: centre, type: architecture, postcode: cb21tt
name: parkside pools, area: centre, type: swimmingpool, postcode: cb11ly
name: pembroke college, area: centre, type: college, postcode: cb21rf
name: people's portraits exhibition at girton college, area: west, type: museum, postcode: cb3ojg
name: primavera, area: centre, type: museum, postcode: cb21sj
name: queens' college, area: west, type: college, postcode: cb39et
name: regency gallery, area: centre, type: museum, postcode: cb11er
name: riverboat georgina, area: north, type: boat, postcode: cb43ax
name: ruskin gallery, area: centre, type: museum, postcode: cb11pt
name: saint barnabas press gallery, area: east, type: museum, postcode: cb13ew
name: saint catharine's college, area: centre, type: college, postcode: cb21rl
name: saint john's college, area: centre, type: college, postcode: cb21tp
name: scott polar museum, area: centre, type: museum, postcode: cb21er
name: scudamores punting co, area: centre, type: boat, postcode: cb21rs
name: sheep's green and lammas land park fen causeway, area: south, type: park, postcode: cb22ad
name: sidney sussex college, area: centre, type: college, postcode: cb23hu
name: soul tree nightclub, area: centre, type: nightclub, postcode: cb23qf
name: tenpin, area: south, type: entertainment, postcode: cb17dy
name: the cambridge corn exchange, area: centre, type: theatre, postcode: cb23qe
name: the cambridge punter, area: centre, type: boat, postcode: cb41as
name: the cherry hinton village centre, area: east, type: mutliple sports, postcode: cb19ej
name: the fez club, area: centre, type: nightclub, postcode: cb23hx
name: the fitzwilliam museum, area: centre, type: museum, postcode: cb21rb
name: the junction, area: south, type: theatre, postcode: cb17gx
name: the man on the moon, area: centre, type: concerthall, postcode: cb12lf
name: the place, area: south, type: nightclub, postcode: cb23hg
name: trinity college, area: centre, type: college, postcode: cb21tq
name: vue cinema, area: centre, type: cinema, postcode: cb11ps
name: wandlebury country park, area: south, type: park, postcode: cb223ae
name: whale of a time, area: west, type: entertainment, postcode: cb238el
name: whipple museum of the history of science, area: centre, type: museum, postcode: cb23rh
name: williams art and antiques, area: centre, type: museum, postcode: cb12lj


USER: Please find a restaurant called Nusha.
SYSTEM: I don't seem to be finding anything called Nusha.  What type of food does the restaurant serve?
USER: I am not sure of the type of food but could you please check again and see if you can find it? Thank you.
SYSTEM: Could you double check that you've spelled the name correctly? The closest I can find is Nandos.
USER: It's not a restaurant, it's an attraction. Nusha.
SYSTEM:"""

In [93]:
response = completion(prompt)

In [94]:
response

"I apologize for the confusion earlier. Unfortunately, I don't have any information on an attraction called Nusha. Could you provide more details on what type of attraction it is or its location so I can assist you better?"